In [2]:
import tensorflow as tf
from tensorflow import keras
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
california_housing = fetch_california_housing()
california_housing

{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ...,
        [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
           39.43      , -121.22      ],
        [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
           39.43      , -121.32      ],
        [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
           39.37      , -121.24      ]]),
 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894]),
 'frame': None,
 'target_names': ['MedHouseVal'],
 'feature_names': ['MedInc',
  'HouseAge',
  'AveRooms',
  'AveBedrms',
  'Population',
  'AveOccup',
  'Latitude',
  'Longitude'],
 'DESCR': '.. _california_housing_dataset:\n

Custom Models

In [4]:
X_train0, X_test, y_train0, y_test = train_test_split(
                 california_housing["data"],
                 california_housing["target"])

In [5]:
sc = StandardScaler()
X_train_s = sc.fit_transform(X_train0)
X_test_s = sc.transform(X_test)

In [6]:
model = keras.models.Sequential([
    keras.layers.Dense(50, activation="relu"),
    keras.layers.Dense(10, activation="relu"),
    keras.layers.Dense(1)
])

In [7]:
model.compile(loss="mean_squared_error",
              optimizer="sgd",
              metrics=["mean_absolute_error"])

Loss

In [8]:
def my_loss(y_true, y_pred):
  error = tf.abs(y_true - y_pred)
  return tf.experimental.numpy.select([error < 0.1, error < 1, error >= 1],
                                      [error, error*2, error**2])

In [9]:
class My_loss(keras.losses.Loss):
  def __init__(self, t1, t2, **kwargs):
    self.t1 = t1
    self.t2 = t2
    super().__init__(**kwargs)
  def call(self, y_true, y_pred):
    error = tf.abs(y_true - y_pred)
    return tf.experimental.numpy.select([error < self.t1,
                                         error < self.t2,
                                         error >= self.t2],
                                        [error, error*2, error**2])
  def get_config(self):
    parent_config = super().get_config()
    return {**parent_config, "t1":self.t1, "t2":self.t2}

In [10]:
def my_loss2(t1, t2):
  def my_loss(y_true, y_pred):
    error = tf.abs(y_true - y_pred)
    return tf.experimental.numpy.select([error < t1, error < t2, error >= t2],
                                        [error, error*2, error**2])
  return my_loss
my_loss_2 = my_loss2(0.1, 1)

In [11]:
model.compile(loss=My_loss(0.1, 1),
              optimizer="sgd",
              metrics=["mean_absolute_error"])

In [12]:
model.fit(X_train_s, y_train0, epochs=5, validation_split=0.15)

Epoch 1/5
412/412 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 2.3428 - mean_absolute_error: 0.9664 - val_loss: 1.7442 - val_mean_absolute_error: 0.8666
Epoch 2/5
412/412 ━━━━━━━━━━━━━━━━━━━━ 0s 846us/step - loss: 2.1657 - mean_absolute_error: 0.8926 - val_loss: 1.7332 - val_mean_absolute_error: 0.8910
Epoch 3/5
412/412 ━━━━━━━━━━━━━━━━━━━━ 0s 748us/step - loss: 1.7701 - mean_absolute_error: 0.8900 - val_loss: 1.7381 - val_mean_absolute_error: 0.8614
Epoch 4/5
412/412 ━━━━━━━━━━━━━━━━━━━━ 0s 759us/step - loss: 1.7375 - mean_absolute_error: 0.8732 - val_loss: 1.7254 - val_mean_absolute_error: 0.8689
Epoch 5/5
412/412 ━━━━━━━━━━━━━━━━━━━━ 0s 790us/step - loss: 1.7146 - mean_absolute_error: 0.8651 - val_loss: 1.7234 - val_mean_absolute_error: 0.8675


In [13]:
model.save("reg_model_My_loss.h5")

In [14]:
model_Myloss = keras.models.load_model("reg_model_My_loss.h5",
                        custom_objects={"My_loss":My_loss})

In [15]:
model_Myloss.predict(X_test_s)

162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 701us/step


array([[2.041447 ],
       [2.1418123],
       [2.060617 ],
       ...,
       [2.1255763],
       [2.0051327],
       [1.9809444]], dtype=float32)

Regularizers

In [16]:
def my_regu(x):
  tf.math.reduce_sum(tf.math.square(x))

Metrics

In [17]:
def r2(y_true, y_pred):
  ss_res = tf.reduce_sum((y_true - y_pred)**2)
  ss_tot = tf.reduce_sum((y_true - tf.reduce_mean(y_true))**2)
  r2 = 1 - ss_res/(ss_tot+1e-6)
  return r2

In [18]:
model.compile(loss=My_loss(0.1, 1),
              optimizer="sgd",
              metrics=[r2])

In [19]:
model.fit(X_train_s, y_train0, epochs=5, validation_split=0.15)

Epoch 1/5
412/412 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1.7410 - r2: -32.4335 - val_loss: 1.7575 - val_r2: -32.3800
Epoch 2/5
412/412 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.7483 - r2: -32.1552 - val_loss: 1.7242 - val_r2: -32.1455
Epoch 3/5
412/412 ━━━━━━━━━━━━━━━━━━━━ 0s 760us/step - loss: 1.7601 - r2: -32.2923 - val_loss: 1.7594 - val_r2: -32.3822
Epoch 4/5
412/412 ━━━━━━━━━━━━━━━━━━━━ 0s 823us/step - loss: 1.7666 - r2: -32.3869 - val_loss: 1.7333 - val_r2: -32.1196
Epoch 5/5
412/412 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.7611 - r2: -32.3154 - val_loss: 1.7385 - val_r2: -33.8864


Layer

In [20]:
class SimpleDense(keras.layers.Layer):

  def __init__(self, units=32):
      super(SimpleDense, self).__init__()
      self.units = units

  def build(self, input_shape):  # Create the state of the layer (weights)
    w_init = tf.random_normal_initializer()
    self.w = tf.Variable(
        initial_value=w_init(shape=(input_shape[-1], self.units),
                             dtype='float32'),
        trainable=True)
    b_init = tf.zeros_initializer()
    self.b = tf.Variable(
        initial_value=b_init(shape=(self.units,), dtype='float32'),
        trainable=True)

  def call(self, inputs):  # Defines the computation from inputs to outputs
      return tf.matmul(inputs, self.w) + self.b

# Instantiates the layer.
linear_layer = SimpleDense(4)

# This will also call `build(input_shape)` and create the weights.
y = linear_layer(tf.ones((2, 2)))
y

<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[ 0.10898118,  0.01374244, -0.02031757,  0.07309394],
       [ 0.10898118,  0.01374244, -0.02031757,  0.07309394]],
      dtype=float32)>

In [21]:
class MyLinear(keras.layers.Layer):

  def __init__(self, units=32):
      super(MyLinear, self).__init__()
      self.units = units

  def build(self, input_shape):  # Create the state of the layer (weights)
    self.w = self.add_weight(shape=(input_shape[-1], self.units),
                             initializer="random_normal",
                             trainable=True)
    self.b = self.add_weight(shape=(self.units,),
                             initializer="random_normal",
                             trainable=True)

  def call(self, inputs):  # Defines the computation from inputs to outputs
      return tf.matmul(inputs, self.w) + self.b

In [22]:
class MyBlock(keras.layers.Layer):
  def __init__(self):
    super(MyBlock, self).__init__()
    self.lin1 = MyLinear(50)
    self.lin2 = MyLinear(100)
    self.lin3 = MyLinear(100)
    self.lin4 = MyLinear(100)

  def call(self, inputs):
    X = self.lin1(inputs)
    X = tf.nn.relu(X)
    X = self.lin2(X)
    X = tf.nn.relu(X)
    X = self.lin3(X)
    X = tf.nn.relu(X)
    X = self.lin4(X)
    loss_ = tf.reduce_mean(X)/2
    self.add_loss(loss_)
    return X

In [23]:
model = keras.models.Sequential([
      MyBlock(),
      MyBlock(),
      keras.layers.Dense(1)
])

In [24]:
model.compile(loss="mean_absolute_error",
              optimizer="sgd",
              metrics=["mean_absolute_error"])

In [25]:
model.fit(X_train_s, y_train0, epochs=5, validation_split=0.15)

Epoch 1/5



412/412 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 1.0085 - mean_absolute_error: 1.0184 - val_loss: 0.8351 - val_mean_absolute_error: 0.8670
Epoch 2/5
412/412 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.8512 - mean_absolute_error: 0.8907 - val_loss: 0.8034 - val_mean_absolute_error: 0.8667
Epoch 3/5
412/412 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.8031 - mean_absolute_error: 0.8750 - val_loss: 0.7656 - val_mean_absolute_error: 0.8658
Epoch 4/5
412/412 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.7841 - mean_absolute_error: 0.8955 - val_loss: 0.7183 - val_mean_absolute_error: 0.8669
Epoch 5/5
412/412 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.7229 - mean_absolute_error: 0.8875 - val_loss: 0.6429 - val_mean_absolute_error: 0.8657


In [26]:
class MyModel(tf.keras.Model):

  def __init__(self):
    super().__init__()
    self.dense1 = tf.keras.layers.Dense(4, activation=tf.nn.relu)
    self.dense2 = tf.keras.layers.Dense(1, activation=tf.nn.softmax)

  def call(self, inputs):
    x = self.dense1(inputs)
    loss_ = tf.reduce_mean(inputs)/2
    self.add_loss(loss_)
    return self.dense2(x)

model = MyModel()

In [27]:
model.compile(loss="mean_absolute_error",
              optimizer="sgd",
              metrics=["mean_absolute_error"])

In [28]:
model.fit(X_train_s, y_train0, epochs=5, validation_split=0.15)

Epoch 1/5
412/412 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1.1744 - mean_absolute_error: 1.1734 - val_loss: 1.1197 - val_mean_absolute_error: 1.1219
Epoch 2/5
412/412 ━━━━━━━━━━━━━━━━━━━━ 0s 811us/step - loss: 1.1589 - mean_absolute_error: 1.1606 - val_loss: 1.1197 - val_mean_absolute_error: 1.1219
Epoch 3/5
412/412 ━━━━━━━━━━━━━━━━━━━━ 0s 851us/step - loss: 1.1682 - mean_absolute_error: 1.1679 - val_loss: 1.1197 - val_mean_absolute_error: 1.1219
Epoch 4/5
412/412 ━━━━━━━━━━━━━━━━━━━━ 0s 948us/step - loss: 1.1669 - mean_absolute_error: 1.1656 - val_loss: 1.1197 - val_mean_absolute_error: 1.1219
Epoch 5/5
412/412 ━━━━━━━━━━━━━━━━━━━━ 0s 988us/step - loss: 1.1534 - mean_absolute_error: 1.1525 - val_loss: 1.1197 - val_mean_absolute_error: 1.1219
